# TensorRider Algorithm
Building regression model with TensorFlow's native APIs.

# Prepare

In [14]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'val.tfrecords'

BATCH_SIZE = 256
EPOCHS = 300

def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image':tf.FixedLenFeature([], tf.string),
            'label':tf.FixedLenFeature([], tf.int64),
        }
    )
    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape([4800])
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    label = tf.cast(features['label'], tf.float32) / 40
    return image, label

def inputs(train, batch_size, num_epochs):
    filename = TRAIN_FILE if train else VAL_FILE
    
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
            [filename], num_epochs=num_epochs
        )
        image, label = read_and_decode(filename_queue)
    
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        min_after_dequeue=1000
    )
    
    return images, sparse_labels
#     return image, label

In [15]:
i,s = inputs(train=True, batch_size=256, num_epochs=300)

In [16]:
print(i.shape, s.shape)

(256, 4800) (256,)


# Define the Dense Layers

In [2]:
x = tf.placeholder(tf.float32, [None, 4800])
W = tf.Variable(tf.zeros([4800, 1]))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, 1])

msqr = tf.reduce_mean(tf.losses.mean_squared_error(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(msqr)

# Train

In [ ]:
sess = tf.InteractiveSession()
for _ in range(EPOCHS):
    batch_xs, batch_ys = inputs(train=True, batch_size=BATCH_SIZE, num_epochs=EPOCHS)
    batch_xs, batch_ys = sess.run([batch_xs, batch_ys])
    _, error = sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    print(_,error)